# **GAN(Generative Adversarial Network)**

진짜 데이터와 구분할 수 없이 비슷한 가짜 데이터를 생성하는 모델 <br>

> **원리**

생성자(generator)와 판별자(discriminator) 두 개의 네트워크가 경쟁하며 학습합니다. <br>생성자는 진짜와 비슷한 가짜를 생성하려 노력하고, 판별자는 더 정확하게 구분하려고 노력합니다. <br>

> **사용처**

Deepfake, 가상현실 게임에서 이미지 생성, 의료 영상 처리, 캐릭터 생성 등등..

> **오늘의 목표**

예전에 보았던 MNIST 숫자 데이터를 GAN으로 생성해봅시다.

[참고](https://github.com/YBIGTA/Deep_learning/blob/master/GAN/2017-07-29-GAN-tutorial-2-MNIST.markdown)

In [11]:
# 모듈을 불러와볼까요?

import tensorflow as tf
from keras.optimizers import Adam
from keras.datasets import mnist

from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

# 데이터도 불러와봅시다.

(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(len(x_train))
print(len(y_train))
print(len(x_test))
print(len(y_test))

# 데이터를 정규화합니다. (대충 ㅎㅎㅎ)
x_train = (x_train.astype(np.float32) - 127.5)/127.5

# 28*28 픽셀데이터인 데이터를 한 줄로 넣기 위해 바꿔줍니다.
mnist_data = x_train.reshape(60000,784)
print(mnist_data.shape)
len(mnist_data)

60000
60000
10000
10000
(60000, 784)


60000

> **신경망 정의**

생성자와 판별자의 신경망을 정해줍니다.



In [7]:
# 생성자 모델 생성 함수
def create_generator():
    generator = tf.keras.models.Sequential([
        # 입력층과 노이즈 벡터의 크기 설정
        tf.keras.layers.Dense(units=256, input_dim=100),
        # LeakyReLU 활성화 함수 적용
        tf.keras.layers.LeakyReLU(0.2),
        # 은닉층 추가
        tf.keras.layers.Dense(units=512),
        # LeakyReLU 활성화 함수 적용
        tf.keras.layers.LeakyReLU(0.2),
        # 출력층 설정
        tf.keras.layers.Dense(units=784, activation='tanh')
    ])

    return generator

# 생성자 모델 생성
g= create_generator()
# 생성자 모델 정보 출력
g.summary()

# 판별자 모델 생성 함수
def create_discriminator():
    discriminator = tf.keras.models.Sequential([
        # 입력층 설정
        tf.keras.layers.Dense(units=512, input_dim=784),
        # LeakyReLU 활성화 함수 적용
        tf.keras.layers.LeakyReLU(0.2),
        # 은닉층 추가
        tf.keras.layers.Dense(units=256),
        # LeakyReLU 활성화 함수 적용
        tf.keras.layers.LeakyReLU(0.2),
        # 출력층 설정
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    # 이진 분류 문제를 위해 binary_crossentropy 손실 함수 설정
    # Adam 최적화 함수 사용
    discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.002, beta_1=0.5))
    return discriminator

# 판별자 모델 생성
d = create_discriminator()
# 판별자 모델 정보 출력
d.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 256)               25856     
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_7 (Dense)             (None, 512)               131584    
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_8 (Dense)             (None, 784)               402192    
                                                                 
Total params: 559,632
Trainable params: 559,632
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_3"
__________________________

/usr/local/lib/python3.9/dist-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


> **GAN 생성함수** 제작

생성자 신경망과 판별자 신경망으로 만듭니다.

In [8]:
def create_gan(discriminator, generator) :
  # Discriminator 모델의 파라미터 업데이트를 막기 위해 trainable을 False로 설정합니다.
  discriminator.trainable = False

  # GAN 모델을 생성합니다.
  # 노이즈 벡터를 입력으로 받아 generator 모델로 가공합니다.
  # 그리고 그 결과를 discriminator 모델의 출력으로 연결합니다.
  gan_input = tf.keras.layers.Input(shape=(100,))
  x = generator(gan_input)
  gan_output = discriminator(x)
  gan = tf.keras.models.Model(inputs=gan_input, outputs=gan_output)

  # GAN 모델을 컴파일합니다.
  # 분류해야하니까 binary_crossentropy 사용합니다.
  gan.compile(loss='binary_crossentropy', optimizer='adam')
  
  return gan

gan = create_gan(d,g)
gan.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential_2 (Sequential)   (None, 784)               559632    
                                                                 
 sequential_3 (Sequential)   (None, 1)                 533505    
                                                                 
Total params: 1,093,137
Trainable params: 559,632
Non-trainable params: 533,505
_________________________________________________________________


> **생성자로 그림 생성**

In [9]:
def plot_generated_images(generator):
    # 입력 잡음 생성
    noise = np.random.normal(loc=0, scale=1, size=[100,100])
    # 생성된 이미지 예측
    generated_images = generator.predict(noise)
    # 이미지 형태 변환
    generated_images = generated_images.reshape(100,28,28)

    # 이미지 시각화
    plt.figure(figsize=(10,10))
    for i in range(generated_images.shape[0]) :
        plt.subplot(10,10, i+1)
        plt.imshow(generated_images[i], interpolation='nearest')
        plt.axis('off')
    plt.tight_layout()

> **학습시키기**

In [10]:
batch_size = 128
epochs = 5000
for e in tqdm(range(epochs)) :
    # 랜덤 노이즈 생성
    noise = np.random.normal(0,1,[batch_size, 100])
    # 노이즈를 이용해 가짜 이미지 생성
    generated_images = g.predict(noise)
    # 실제 이미지에서 랜덤하게 배치를 가져옴
    image_batch = mnist_data[np.random.randint(low=0, high= mnist_data.shape[0], size = batch_size)]
    
    # 실제 이미지와 가짜 이미지를 결합
    x = np.concatenate([image_batch, generated_images])

    # Discriminator의 라벨 생성
    y_dis = np.zeros( 2* batch_size)
    y_dis[:batch_size] = 1

    # Discriminator 학습
    d.trainable = True
    d.train_on_batch(x, y_dis)
    
    # Generator 학습
    noise = np.random.normal(0,1, [batch_size, 100])
    y_gen = np.ones(batch_size)
    d.trainable = False
    gan.train_on_batch(noise, y_gen)
    
    # 1000 에폭마다 생성된 이미지 시각화
    if e==0 or e % 1000 == 0 :
        plot_generated_images(g)  

Output hidden; open in https://colab.research.google.com to view.